# Call Tensorboard
<p> Put name of logs file (obtained from hyperparameter tuning) after --logdir </p>

In [2]:
%load_ext tensorboard
%tensorboard --logdir ktuner/logs_pdac_once_ntl_3/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Code to make custom plots
<p> Code to make custom plots from specific hyperparameter tuning trials </p>

In [9]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np

def get_scalar_run_tensorboard(tag, filepath):
    values,steps = [],[]
    for e in tf.compat.v1.train.summary_iterator(filepath):
        if len(e.summary.value)>0: #Skip first empty element
            if e.summary.value[0].tag==tag:
                tensor = (e.summary.value[0].tensor)
                value,step = (tf.io.decode_raw(tensor.tensor_content,tf.float32)[0].numpy(),e.step)
                values.append(value)
                steps.append(step)
    return values,steps


train_values = []
train_steps = []
val_values = []
val_steps = []

for folds in range(5):
    folder_name = "ktuner/logs_pdac_11/7/fold_" + str(folds + 1)
    for file in sorted(os.listdir(folder_name + "/train/")):
        filename = folder_name + "/train/" + file
        val,st = get_scalar_run_tensorboard("epoch_loss",filename)
        train_values.append(val)
        train_steps.append(st)
    for file in sorted(os.listdir(folder_name + "/validation/")):
        filename = folder_name + "/validation/" + file
        val,st = get_scalar_run_tensorboard("epoch_loss",filename)
        val_values.append(val)
        val_steps.append(st)
        

final_losses = []
early_stopping_val = 80

for fold in val_values:
    final_losses.append(fold[-early_stopping_val])

print(final_losses)
print(np.std(final_losses))


# fig = plt.figure(figsize=(12, 6))
# ax = fig.add_subplot(121)
# ax2 = fig.add_subplot(122)
# #ax2.set_prop_cycle(color=['red','red','black', 'black','yellow', 'yellow','green', 'green','blue', 'blue'])
# ax2.set_prop_cycle(color=['red','red','black', 'black','yellow', 'yellow','green', 'green','blue', 'blue','pink','pink','cyan','cyan','darkviolet','darkviolet','chartreuse','chartreuse','navajowhite','navajowhite'])
# i = 1

# minimum_x = 0
# maximum_x = 2300
# minimum_y = 1.0
# maximum_y = 1.6
        
# for train_values,train_steps,val_values,val_steps in zip(train_values,train_steps,val_values,val_steps):

#     ax.plot(train_steps,train_values, color="blue")
#     ax.plot(val_steps,val_values, color="red")
#     ax.axis(xmin=minimum_x,xmax=maximum_x,ymin=minimum_y,ymax=maximum_y)
#     ax.legend(['train','val'])
#     ax.set_xlabel('Epochs')
#     ax.set_ylabel('Loss')
#     ax2.plot(train_steps,train_values)
#     ax2.plot(val_steps,val_values)
#     ax2.axis(xmin=minimum_x,xmax=maximum_x,ymin=minimum_y,ymax=maximum_y)
#     #ax2.legend(['fold 1','fold 2','fold 3','fold 4','fold 5'])
#     ax2.legend(['loop 1','loop 2','loop 3','loop 4','loop 5','loop 6','loop 7','loop 8','loop 9','loop 10'])
#     ax2.set_xlabel('Epochs')
#     ax2.set_ylabel('Loss')


[1.4422638, 1.2045612, 1.2879964, 1.3790606, 1.2167698]
0.09217547
